In [52]:
# Import dependencies
import findspark
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
import pandas as pd

In [58]:
# Initialize findspark
findspark.init()

# Initialize the spark session
spark = SparkSession.builder.appName("SK_model").getOrCreate()

In [64]:
# Load in the pre-processed data
path = "../Resources/filtered_encoded_dataset.csv"
spark.sparkContext.addFile(path)
df = spark.read.csv(SparkFiles.get("filtered_encoded_dataset.csv"), sep=",", header=True, inferSchema=True)
#df = spark.read.csv("../Resources/filtered_encoded_dataset.csv", sep=",", header=True, inferSchema=True)
df.show()
df.write.csv('test.csv', mode='overwrite')

+---+-----------------+--------------------+----------+--------+------------+------+--------+----+-----------+------------+----------------+--------+-------+-------+------------------+----------------+----------------+----------------+----------------+----------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|_c0|   primary_artist|          track_name|popularity|explicit|danceability|energy|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|      duration_min|time_signature_0|time_signature_1|time_signature_3|time_signature_4|time_signature_5|key_0|key_1|key_2|key_3|key_4|key_5|key_6|key_7|key_8|key_9|key_10|key_11|num_artists_binned_1|num_artists_binned_2|num_artists_binned_3|num_artists_binned_4|num_a

In [57]:
# Remove index, primary_artist, and track_name from the dataset
df = df.drop("_c0", "primary_artist", "track_name")
df.show()

+----------+--------+------------+------+--------+----+-----------+------------+----------------+--------+-------+-------+------------------+----------------+----------------+----------------+----------------+----------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|popularity|explicit|danceability|energy|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|      duration_min|time_signature_0|time_signature_1|time_signature_3|time_signature_4|time_signature_5|key_0|key_1|key_2|key_3|key_4|key_5|key_6|key_7|key_8|key_9|key_10|key_11|num_artists_binned_1|num_artists_binned_2|num_artists_binned_3|num_artists_binned_4|num_artists_binned_5|num_artists_binned_6|track_genre_0|track_genre_1|track_genre_2|track_g

In [67]:
# Convert the data to a pandas Dataframe
p_df = pd.read_csv("../Resources/filtered_encoded_dataset.csv")
p_df.to_csv("test.csv")
p_df['popularity'].value_counts().sort_values()

popularity
99         1
100        2
95         5
98         7
94         7
96         7
97         8
92         9
91        10
93        12
89        27
90        30
88        43
87        49
86        53
85       101
84       103
83       156
82       158
81       165
80       246
79       252
78       254
77       314
4        377
76       393
75       417
6        419
7        455
73       467
72       485
74       486
9        522
8        538
14       539
3        570
71       576
5        592
15       597
70       625
69       632
13       691
68       702
10       723
11       787
12       799
67       825
66       893
64       929
65       952
16       964
2       1025
63      1027
61      1054
62      1079
17      1115
31      1200
60      1251
30      1267
33      1308
32      1314
59      1330
53      1351
29      1395
58      1396
34      1408
54      1435
55      1443
18      1452
57      1456
35      1459
52      1470
28      1493
27      1514
51      1516
36      1523
5

In [43]:

# Convert data types
conversions = {
    "popularity": int,
    'explicit': int, 
    'danceability': float, 
    'energy': float,
    'loudness': float,
    'mode': int,
    'speechiness': float,
    "acousticness": float,
    'instrumentalness': float,
    'liveness': float,
}
p_df = p_df.astype(conversions)

p_df.dtypes


ValueError: invalid literal for int() with base 10: ' Pt. 1) [Music from the Original TV Series]"': Error while type casting for column 'popularity'

In [41]:
# Bin the popularity so it can be predicted (high/ medium/ low)
## Check a histogram of popularity
p_df.hist(column='popularity')

ValueError: hist method requires numerical or datetime columns, nothing to plot.

In [40]:
# Separate the prediction (song popularity) from the rest of the features
y = p_df.popularity.values
X = p_df.drop(columns="popularity")

# Split the training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.